# Imitation Learning 모델 1
---
## Description
* vanilla DAgger 모델

## todo
* 모듈 2, 모듈 3 구현

* pickle 등으로 로컬에 데이터 저장

## 링크
* [PEP8 코딩준수](https://kongdols-room.tistory.com/18)
* [openCV 가이드](https://opencv-python.readthedocs.io/en/latest/doc/02.videoStart/videoStart.html)
* [프로그레스 바 구현](https://wikidocs.net/13977)

## 1. 학습데이터 생성 모듈

In [36]:
import cv2
import numpy as np
import copy

In [2]:
class DataCollector:
    def __init__(self):
        pass
    

In [166]:
class DataProducer:
    def __init__(self, width=320, height=240):
        self.observation = [] # 전처리된 영상 데이터 np 원본
        self.label = [] # 라벨링 데이터, observation 과 같은 크기
        self.width = width
        self.height = height
        print('> data producer 생성. ')
        print('> 가로 : {}, 세로 : {}'.format(width, height))
        print('-'*50)
        
    def __repr__(self):
        return '가로 : {}, 세로 : {}'.format(self.width, self.height)
    
    def produce_observation(self, file=None):
        # observation 을 생성하는 메서드
        
        if(file == None):
            # 재생할 원본영상이 없으면 실시간으로 카메라 영상 재생
            # 여기서는 노트북의 로컬카메라
            cap = cv2.VideoCapture(0)
            cap.set(3, self.width)
            cap.set(4, self.height)
        else:
            # 파일로부터 원본영상을 받아 재생
            # 처음부터 전처리한 영상을 안만드는 이유는
            # 하나의 원본으로부터 다양한 전처리 작업을 한 결과을 얻기 위함
            cap = cv2.VideoCapture(file)
        
        while(cap.isOpened()):
            ret, frame = cap.read()
            if(ret):
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                ret_bin, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
                #cv2.imshow('frame', gray)
                cv2.imshow('binary', binary)
                self.observation.append(binary)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        cap.release()
        cv2.destroyAllWindows()
        print('> 원본 프레임 수 : {}'.format(len(self.observation)))
        print('-'*50)
    
    def produce_label(self, video, delay=10):
        # 외부에서 만든 영상파일을 받아 수동으로 라벨링을 하는 메서드
        # 수동으로 라벨링하는 과정을 observation 구하는 것과 분리함.
        # 다양한 라벨링을 테스트하기 위함
        self.label = []
        command = [0,1,0]
        cap = cv2.VideoCapture(video)        
        while(cap.isOpened()):
            ret, frame = cap.read()
            if(ret):
                key = cv2.waitKey(delay) & 0xFF
                if(key == ord('q')):
                    break
                elif(key == ord('a')):
                    command = [1,0,0]
                elif(key == ord('d')):
                    command = [0,0,1]
                elif(key == ord('w')):
                    command = [0,1,0]
                
                self.label.append(command)
                cv2.putText(frame, 'command : {}'.format(command), (10, 30), cv2.FONT_HERSHEY_COMPLEX, .7, (0,0,0), 2)
                cv2.imshow('debug', frame)
            else:
                break
        cap.release()
        cv2.destroyAllWindows()
        print('> 라벨링 완료 프레임 수 : {}'.format(len(self.label)))
        print('-'*50)
    
    def get_observation(self):
        # 외부에서 영상을 만들기위해 원본을 제공
        video_data = copy.deepcopy(self.observation)
        print('> 전처리 영상 복사완료 - 프레임 수: {}'.format(len(self.observation)))
        print('-'*50)
        return video_data
    
    def get_label(self):
        # 라벨 데이터를 제공하는 메서드
        label_data = copy.deepcopy(self.label)
        print('> 라벨 데이터 복사완료 - 라벨 수: {}'.format(len(self.label)))
        return label_data
    
    def get_database(self):
        # observation - label 데이터 쌍을 data 학습데이터로 생성하는 메서드
        print('> 영상 프레임 수 : {}, 라벨 프레임 수 : {}, 부족한 라벨 수: {}'.format(len(self.observation), len(self.label), len(self.observation) - len(self.label)))
        print('> {} 프레임 학습 데이터 생성완료.'.format(len(self.label)))
        print('-'*50)
        return zip(self.observation[:len(self.label)], self.label)

In [167]:
dp = DataProducer() # 학습데이터 생성기 객체 생성

> data producer 생성. 
> 가로 : 320, 세로 : 240
--------------------------------------------------


In [168]:
dp.produce_observation() # raw 영상으로부터 전처리영상 생성

> 원본 프레임 수 : 38
--------------------------------------------------


In [169]:
fourcc = cv2.VideoWriter_fourcc(*'MP42')
fps = 25.0
out = cv2.VideoWriter('output.avi', fourcc, fps, (320,240), 0) # 세밀한 컨트롤을 위해 프레임 늘리기 고려

In [170]:
# 전처리된 영상원본을 동영상파일로 저장
copied_observation = dp.get_observation()
for f in range(len(copied_observation)):
    frame = copied_observation[f]
    out.write(frame)
out.release()

> 전처리 영상 복사완료 - 프레임 수: 38
--------------------------------------------------


In [175]:
dp.produce_label('output.avi', delay=100) # 만들어진 동영상파일을 가지고 수동으로 라벨링하여 라벨을 프레임별로 따로 저장

> 라벨링 완료 프레임 수 : 24
--------------------------------------------------


In [176]:
data = list(dp.get_database()) # observation - label 모음 학습데이터를 획득한다.
print(len(data))
for d in data:
    ob, lb = d
    #print(ob, lb)

> 영상 프레임 수 : 38, 라벨 프레임 수 : 24, 부족한 라벨 수: 14
> 24 프레임 학습 데이터 생성완료.
--------------------------------------------------
24


## 2. 신경망 학습 모듈

## 3. 실시간 Data 보강 모듈